# Purpose

This notebook largely serves to allow me to sift through all of the chargemasters and metadata generated via the work already done in [this wonderful repo](https://github.com/vsoch/hospital-chargemaster) (from which I forked my repo). 

Based upon the originating repo's own README, there's at least some data collection that still needs to be done (e.g. [data from hospitalpriceindex.com](https://search.hospitalpriceindex.com/hospital/Barnes-Jewish-Hospital/5359?page=1) has to be scraped but they're denying IP addresses that try to do so). However, if any of that gets done in here, it won't be until after I've sifted through the current material to make sure I have a handle on what data are already available. 

It's fairly plausible that I'll then attempt to combine it all into a single sqlite or MongoDB database that can subsequently be analyzed. But I'm getting ahead of myself - first to figure out what we have to work with!

# Background

*Assume everything in this cell is quoted directly from the originating repo README, albeit with some extra content removed for the purposes of streamlining. Anything in italics like this should be assumed to be editorial additions by me.*

**From the original README:**

## Get List of Hospital Pages
We have compiled a list of hospitals and links in the [hospitals.tsv](hospitals.tsv) 
file, generated via the [0.get_hospitals.py](0.get_hospitals.py) script *which pulls these data from [a Quartz article](https://qz.com/1518545/price-lists-for-the-115-biggest-us-hospitals-new-transparency-law/) detailing ~115 hospital URLs from which the authors were able to find chargemasters in one form or another*. 

The file includes the following variables, separated by tabs:

 - **hospital_name** is the human friendly name
 - **hospital_url** is the human friendly URL, typically the page that includes a link to the data.
 - **hospital_id** is the unique identifier for the hospital, the hospital name, in lowercase, with spaces replaced with `-`
   
## Organize Data

Each hospital has records kept in a subfolder in the [data](data) folder. Specifically,
each subfolder is named according to the hospital name (made all lowercase, with spaces 
replaced with `-`). If a subfolder begins with an underscore, it means that I wasn't
able to find the charge list on the hospital site (and maybe you can help?) 
Within that folder, you will find:

 - `scrape.py`: A script to scrape the data
 - `browser.py`: If we need to interact with a browser, we use selenium to do this.
 - `latest`: a folder with the last scraped (latest data files)
 - `YYYY-MM-DD` folders, where each folder includes:
   - `records.json` the complete list of records scraped for a particular data
   - `*.csv` or `*.xlsx` or `*.json`: the scraped data files.

## Parsing
This is likely one of the hardest steps. I wanted to see the extent to which I could
create a simple parser that would generate a single TSV (tab separted value) file
per hospital, with minimally an identifier for a charge, and a price in dollars. If
provided, I would also include a description and code:

 - **charge_code**
 - **price**
 - **description**
 - **hospital_id**
 - **filename**

Each of these parsers is also in the hospital subfolder, and named as "parser.py." The parser would output a data-latest.tsv file at the top level of the folder, along with a dated (by year `data-<year>.tsv`). At some point
I realized that there were different kinds of charges, including inpatient, outpatient, DRG (diagnostic related group) and others called
"standard" or "average." I then went back and added an additional column
to the data:

 - **charge_type** can be one of standard, average, inpatient, outpatient, drg, or (if more detail is supplied) insured, uninsured, pharmacy, or supply. This is not a gold standard labeling but a best effort. If not specified, I labeled as standard, because this would be a good assumption.

# Exploratory Data Analysis (EDA)

OK, I think I have a handle on this, let's take a look at the data, starting with the hospital metadata.

It may be a bit confusing for anyone following along at home, but note that I had already started my own effort to download a bunch of these chargemasters in a far more manual approach than what @vsoch did with `BeautifulSoup` and all. As a result, I may be comparing her dataset at times in this notebook to the one I was developing. Mine was never going to have automated updates like hers however, hence why I'm deferring to her repo and data over my own (while mine may be a bit more comprehensive in terms of number of hospitals, I'd rather the data be up to date as much as possible). 

That said, as of this writing, I have approximately 600+ unique hospitals included in my chargemaster index, so I'm going to keep an eye out during my EDA to see if my more manual approach may still be useful.

## Metadata

In [1]:
#Make sure any changes to custom packages can be reflected immediately 
#in the notebook without kernel restart
import autoreload
%load_ext autoreload
%autoreload 2

In [2]:
#Import the hospital metadata

import pandas as pd
metadata = pd.read_csv('hospitals.tsv', delimiter = r'\t')
metadata

/Users/emigre459/anaconda3/envs/HospitalPricing/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,hospital_name,hospital_id,hospital_url
0,Atlanticare Regional Medical Center,atlanticare-regional-medical-center,https://www.atlanticare.org/patients-and-visit...
1,Aurora Health Care Metro Inc.,aurora-health-care-metro-inc.,https://www.aurorahealthcare.org/patients-visi...
2,Baptist Health System (San Antonio),baptist-health-system-(san-antonio),https://www.baptisthealthsystem.com/for-patien...
3,Baptist Hospital (Miami),baptist-hospital-(miami),https://baptisthealth.net/en/facilities/baptis...
4,Baptist Medical Center (Jacksonville),baptist-medical-center-(jacksonville),https://www.baptistjax.com/patient-info/billin...
5,Barnes Jewish Hospital,barnes-jewish-hospital,https://www.bjc.org/For-Patients-Billing-Visit...
6,Brigham and Womens Hospital,brigham-and-womens-hospital,https://www.partners.org/for-patients/Patient-...
7,California Pacific Medical Center,california-pacific-medical-center,https://www.sutterhealth.org/cpmc/for-patients...
8,California Pacific Medical Center R.K. Davies ...,california-pacific-medical-center-r.k.-davies-...,https://www.sutterhealth.org/for-patients/heal...
9,Carolinas Medical Center,carolinas-medical-center,https://atriumhealth.org/for-patients-visitors...


In [3]:
#Do these data include the huge CA chargemaster dataset on oshpd.ca.gov?
metadata[metadata['hospital_url'].str.contains('oshpd.ca.gov')]

,hospital_name,hospital_id,hospital_url
34,Loma Linda University Medical Center,loma-linda-university-medical-center,https://oshpd.ca.gov/data-and-reports/cost-tra...
36,Lucile Packard Childrens Hospital,lucile-packard-childrens-hospital,https://oshpd.ca.gov/data-and-reports/cost-tra...
66,Ronald Reagan UCLA Medical Center,ronald-reagan-ucla-medical-center,https://oshpd.ca.gov/data-and-reports/cost-tra...
76,Stanford Hospitals and Clinics,stanford-hospitals-and-clinics,https://oshpd.ca.gov/data-and-reports/cost-tra...


*Interesting, it looks like this dataset may not include the full OSHPD chargemaster list.* I find that unlikely however, as @vsoch makes it clear in another part of her README that she built the `oshpd-ca` scraper and found 795+ hospitals in that dataset. **This suggests that this metadata file may not be complete, as it's really just an inventory of the links from the aforementioned Quartz article, and not necessarily a full accounting of the hospitals contained in the dataset.**

## Reading in the Tabulated Data

OK, so the metadata table wasn't super useful (investigating `data/oshpd-ca` confirmed that indeed there are far more files in this repo than hospitals listed in `hospitals.tsv`), but there are **a lot** of files to plow through here! And vsoch was kind enough to try and compile them whenever appropriate in the various hospital/site-specific folders within `data` as `data-latest[-n].tsv` (`-n` indicates that, if the file gets above 100 MB, it's split into `data-latest-1.tsv`, `data-latest-2.tsv`, etc. to avoid going over the GitHub per-file size limit).

Let's try to parse all of these TSV files into a single coherent DataFrame! The entire `data` folders is less than 4 GB, and I'm confident that more than half of that is individual XLSX/CSV files, so I think this should be something we can hold in memory easily enough.

...still, we'll use some tricks (e.g. making the sub-dataframes as a generator instead of a list) to ensure optimal memory usage, just to be safe.

In [4]:
# Search through the data/hospital-id folders for data-latest[-n].tsv files
# so you can concatenate them into a single DataFrame
from glob import glob, iglob

for name in iglob('data/*/data-latest*.tsv'):
    print(name)

data/university-of-virginia-medical-center/data-latest.tsv
data/university-hospitals-case-medical-center/data-latest.tsv
data/montefiore-medical-center/data-latest.tsv
data/swedish-medical-center/data-latest.tsv
data/temple-university-hospital/data-latest.tsv
data/rush-university-medical-center/data-latest.tsv
data/long-island-jewish-medical-center/data-latest.tsv
data/advent-health/data-latest.tsv
data/atlanticare-regional-medical-center/data-latest.tsv
data/northshore-university-health-system/data-latest.tsv
data/university-of-iowa-hospitals-and-clinics/data-latest.tsv
data/orlando-health/data-latest.tsv
data/north-shore-university-hospital/data-latest.tsv
data/jackson-memorial/data-latest.tsv
data/carolinas-medical-center/data-latest.tsv
data/st.-luke’s-hospital-(san-francisco)/data-latest.tsv
data/memorial-regional-hospital/data-latest.tsv
data/geisinger-medical-center/data-latest.tsv
data/aurora-health-care-metro-inc./data-latest.tsv
data/uc-irvine-medical-center/data-latest.tsv
d

In [ ]:
# Setup the full dataframe using iterators/generators to save on memory
all_files = iglob('data/*/data-latest*.tsv')
individual_dfs = (pd.read_csv(f, delimiter = '\t', 
                              low_memory = False,
                             thousands = ',') for f in all_files)

df = pd.concat(individual_dfs, ignore_index=True)

In [16]:
df.info(memory_usage = 'deep', verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9907857 entries, 0 to 9907856
Data columns (total 6 columns):
charge_code    6563312 non-null object
price          8735057 non-null object
description    8745419 non-null object
hospital_id    9888293 non-null object
filename       9907857 non-null object
charge_type    9907857 non-null object
dtypes: object(6)
memory usage: 3.8 GB


In [8]:
df.head()

,charge_code,price,description,hospital_id,filename,charge_type
0,NaN,1980,Adult semi-private (up to two patients) room a...,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
1,NaN,1980,Adult room and care in the Transitional Care H...,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
2,NaN,2310,Adult room and care on the Labor & Delivery unit,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
3,NaN,1980,Pediatric room and care,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
4,NaN,2448,Psychiatric room and care,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard


**Not bad! At first I was a bit worried when I looked at this initial file in Excel, but Excel improperly interprets the thousands comma on prices as a new field delimiter.** This might work after all!

In [10]:
df.tail()

,charge_code,price,description,hospital_id,filename,charge_type
9907852,NaN,113699.36,983 EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,Winter Haven Hospitals,copy-of-msdrg-list-01-15-19.xlsm,drg
9907853,NaN,91145.59,987 NON-EXTENSIVE O.R. PROC UNRELATED TO PRINC...,Winter Haven Hospitals,copy-of-msdrg-list-01-15-19.xlsm,drg
9907854,NaN,101386.94,988 NON-EXTENSIVE O.R. PROC UNRELATED TO PRINC...,Winter Haven Hospitals,copy-of-msdrg-list-01-15-19.xlsm,drg
9907855,NaN,4701.0,989 NON-EXTENSIVE O.R. PROC UNRELATED TO PRINC...,Winter Haven Hospitals,copy-of-msdrg-list-01-15-19.xlsm,drg
9907856,NaN,67846.66,999 UNGROUPABLE,Winter Haven Hospitals,copy-of-msdrg-list-01-15-19.xlsm,drg


In [11]:
# Push out unoptimized CSV copy of dataframe so we don't need it to perpetually be in memory

df.to_csv('data/all_hospitals-latest.csv')

## Optimize the DataFrame

This dataframe is quite large (almost 4GB in memory!) and that's likely to cause all sorts of problems when it comes time to do analysis. So we'll need to make sure we understand the nature of each column's data and then optimize for it (after potentially correcting for things like strings that are actually numbers).

To start with, let's check out the nature of the data in each column:

1. How often do charge codes exist?
2. We'll check, but likely that description exists for most
3. Other than decimal numbers, what values are in price?
4. What are all of the different charge type values, and which ones can we filter out (e.g. drg)?
    * Do charge codes get embedded in descriptions a lot, like what we see in df.tail()? Or is this something that is only present for non-standard charge_type?

In [10]:
# In case you need to reinitialize df after experimenting...

df = pd.read_csv('data/all_hospitals-latest.csv', index_col = 0)

/Users/emigre459/anaconda3/envs/HospitalPricing/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/emigre459/anaconda3/envs/HospitalPricing/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
df.head()

,charge_code,price,description,hospital_id,filename,charge_type
0,NaN,1980,Adult semi-private (up to two patients) room a...,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
1,NaN,1980,Adult room and care in the Transitional Care H...,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
2,NaN,2310,Adult room and care on the Labor & Delivery unit,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
3,NaN,1980,Pediatric room and care,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard
4,NaN,2448,Psychiatric room and care,university-of-virginia-medical-center,university-of-virginia-medical-center.csv,standard


In [ ]:
# First, let's get a better handle on unique values, so we can figure out what fields make
# sense as categoricals and what ones definitely don't

unique_vals = pd.DataFrame(df.nunique()).rename(columns = {0: 'number of unique values'})
unique_vals['fraction of all non-null values'] = round(unique_vals['number of unique values'] / len(df.dropna()), 2)
unique_vals['make categorical?'] = unique_vals['fraction of all non-null values'] < 0.5
unique_vals['data type'] = df.dtypes
unique_vals

In [18]:
unique_vals['fraction of all non-null values'] = round(unique_vals['number of unique values'] / len(df.dropna()), 2)
unique_vals

,number of unique values,fraction of all values,make categorical?,data type,fraction of all non-null values
charge_code,1878137,0.19,True,object,0.29
price,783989,0.08,True,object,0.12
description,2477488,0.25,True,object,0.38
hospital_id,379,0.00,True,object,0.00
filename,705,0.00,True,object,0.00
charge_type,12,0.00,True,object,0.00


**Interesting. There are so many records with repeat information that we can change the dtype of pretty much all of them to be categorical.** Here's what we're going to do:

1. `charge_code`, `price`, and `description`: I'm not going to convert these to categoricals
    * `charge_code` and `description`: while from a data perspective these would likely be better handled in memory by making anything categorical that has so few unique values that the count of them is less than 50% of the count of all non-null rows, it doesn't make sense to make these fields categoricals, as that implies they are drawing from a common data reference. **That's simply not the case.** 
        * Given that two different hospitals can have the charge code `1001` refer to two totally different procedures/consumables, there's no reason to add confusion by treating these in the dataset like they have the same meaning. 
        * The same logic goes for the description field (although that one has me scratching my head a bit, as I'd expect it to be a bit more free text in nature and thus not likely to have repeated values)
    * `price`: this should be a continuous variable, not a categorical one!
2. `hospital_id`, `filename`, and `charge_type`: these are classic examples of categorical variables and we should convert them.

In [29]:
df['charge_type'].dtype

dtype('O')

In [30]:
# Make categorical columns where appropriate
cat_cols = ['hospital_id', 'filename', 'charge_type']

for col in cat_cols:
    df.loc[:, col] = df.loc[:, col].astype('category')

df['charge_type'].cat.categories

Index(['drg', 'inpatient', 'insurance', 'insured-inpatient',
       'insured-outpatient', 'outpatient', 'pharmacy', 'standard', 'supply',
       'uninsured', 'uninsured-inpatient', 'uninsured-outpatient'],
      dtype='object')

In [32]:
df.info(memory_usage = 'deep', verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9907857 entries, 0 to 9907856
Data columns (total 6 columns):
charge_code    6563312 non-null object
price          8735057 non-null object
description    8745419 non-null object
hospital_id    9888293 non-null category
filename       9907857 non-null category
charge_type    9907857 non-null category
dtypes: category(3), object(3)
memory usage: 1.7 GB


**Nice! We cut the memory usage in half!** OK, on to less obvious optimizing!

In [33]:
# What do our missing values look like? How sparse are these data?

missing = pd.DataFrame(df.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(df),2)
missing.sort_values('total missing', ascending = False)
missing

,total missing,percent missing
charge_code,3344545,0.34
price,1172800,0.12
description,1162438,0.12
hospital_id,19564,0.00
filename,0,0.00
charge_type,0,0.00


**Looks like we have description text for all but 12% of the data.** Not bad.

In [34]:
# How often do charge codes exist?
df['charge_code'].describe()

count     6563312.0
unique    1878137.0
top          1001.0
freq        67966.0
Name: charge_code, dtype: float64

In [15]:
df['charge_code'].value_counts()

1001.0          67966
C1776           39701
301.0           28342
361.0           28316
0272            20236
320.0           19351
27800037.0      14734
306.0           11891
C1713           11710
0278            11686
302.0           10656
27800002        10043
J3490            9693
510.0            8752
278.0            8161
360.0            7689
270.0            7647
0761             7466
250.0            7199
0360             6598
27800072.0       6391
0270             5662
63700001         5551
272.0            5534
0301             5509
305.0            5191
27200178         5028
27200030         4821
SCREW            4725
272.0            4597
                ...  
356478              1
441364              1
812532              1
701014236348        1
CS122               1
409346              1
5355712.0           1
8890382.0           1
30601866            1
364056              1
809721              1
165064              1
5355715.0           1
50748250            1
259297    

**Quite a few different charge codes, with lots of different possible values.** Given that charge codes are likely a somewhat-random "unique" identifiers (air quotes because I'm suspicious of any large entity's data management practices until that suspicion is proven unwarranted). Nothing to see here.

*OK, let's get to the meat of it, the thing that's actually most interesting (arguably): the price!*

In [35]:
# Separate out only the price rows that are non-numeric

df['price'].dropna().apply(type).value_counts()

<class 'str'>      4673821
<class 'float'>    4061236
Name: price, dtype: int64

**So about half of the non-null prices are `str` type.** Now we need to figure out what those strings actually are.

In [36]:
# Filter out non-string prices

df.loc[df['price'].apply(type) == str,'price']

655360           0.0
655361           0.0
655362           0.0
655363           0.0
655364           0.0
655365           0.0
655366           0.0
655367           0.0
655368           0.0
655369           0.0
655370           0.0
655371           0.0
655372           0.0
655373           0.0
655374           0.0
655375           0.0
655376           0.0
655377           0.0
655378           0.0
655379           0.0
655380           0.0
655381           0.0
655382           0.0
655383           0.0
655384           0.0
655385           0.0
655386           0.0
655387           0.0
655388           0.0
655389           0.0
             ...    
9907827     42609.08
9907828     17784.24
9907829      53461.5
9907830     29037.21
9907831     41134.92
9907832     44698.45
9907833     18743.77
9907834     57341.27
9907835     17131.68
9907836     25042.51
9907837    120471.76
9907838       1393.0
9907839     22365.63
9907840     24346.83
9907841     11872.24
9907842     81348.51
9907843     6

**Huh, how odd. These strings all look like regular old float values.** Why is pandas importing them as strings? Let's force the column to be float type and then we'll see how our missing values change (since we can force non-numeric strings to be `NaN`)

In [37]:
# Convert price to be float type, then see if missing['total_missing'] changes

missing_price_initial = missing.loc['price','total missing']

delta = pd.to_numeric(df['price'], errors = 'coerce').isnull().sum() - missing_price_initial
delta

294073

**OK, so we see about 300K of the 4.67M string values become `NaN` when we do this numeric conversion. That's not ideal.** Looks like we're losing a lot of data with this conversion, is there a better way? Or should we just consider this an acceptable loss?

In [ ]:
# How to filter so we can see a sample of the ~300K true non-float strings in price
# so we can get an idea as to how to deal with them?

